In [12]:
import numpy as np
import json
import re
import tensorflow as tf
import random
import spacy
nlp = spacy.load('en_core_web_sm')

Import json file

In [13]:
with open('Intent.json') as f:
    intents = json.load(f)

Preprocessing data

In [14]:
def preprocessing(line):
    line = re.sub(r'[^a-zA-z.?!\']', ' ', line)
    line = re.sub(r'[ ]+', ' ', line)
    return line

In [15]:
# get text and intent title from json data
inputs, targets = [], []
classes = []
intent_doc = {}

for intent in intents['intents']:
    if intent['intent'] not in classes:
        classes.append(intent['intent'])
    if intent['intent'] not in intent_doc:
        intent_doc[intent['intent']] = []
        
    for text in intent['text']:
        inputs.append(preprocessing(text))
        targets.append(intent['intent'])
        
    for response in intent['responses']:
        intent_doc[intent['intent']].append(response)

In [16]:
def tokenize_data(input_list):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<unk>')
    
    tokenizer.fit_on_texts(input_list)
    
    input_seq = tokenizer.texts_to_sequences(input_list)

    input_seq = tf.keras.preprocessing.sequence.pad_sequences(input_seq, padding='pre')
    
    return tokenizer, input_seq

# preprocess input data
tokenizer, input_tensor = tokenize_data(inputs)

In [17]:
def create_categorical_target(targets):
    word={}
    categorical_target=[]
    counter=0
    for trg in targets:
        if trg not in word:
            word[trg]=counter
            counter+=1
        categorical_target.append(word[trg])
    
    categorical_tensor = tf.keras.utils.to_categorical(categorical_target, num_classes=len(word), dtype='int32')
    return categorical_tensor, dict((v,k) for k, v in word.items())

# preprocess output data
target_tensor, trg_index_word = create_categorical_target(targets)

In [18]:
print('input shape: {} and output shape: {}'.format(input_tensor.shape, target_tensor.shape))

input shape: (144, 9) and output shape: (144, 22)


Building the model

In [19]:
# hyperparameters
epochs=100
vocab_size=len(tokenizer.word_index) + 1
embed_dim=512
units=128
target_length=target_tensor.shape[1]

In [20]:
# build RNN Model with tensorflow
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embed_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units, dropout=0.2)),
    tf.keras.layers.Dense(units, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(target_length, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(lr=1e-2)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 512)         66048     
                                                                 
 bidirectional_1 (Bidirecti  (None, 256)               656384    
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 22)                2838      
                                                                 
Total params: 758166 (2.89 MB)
Trainable params: 758166 (2.89 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [21]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)

# train the model
model.fit(input_tensor, target_tensor, epochs=epochs, callbacks=[early_stop])
model.save("model.h5")

Epoch 1/100
5/5 [==============================] - 5s 39ms/step - loss: 3.0936 - accuracy: 0.0486
Epoch 2/100
5/5 [==============================] - 0s 41ms/step - loss: 3.0551 - accuracy: 0.1528
Epoch 3/100
5/5 [==============================] - 0s 43ms/step - loss: 3.0236 - accuracy: 0.1250
Epoch 4/100
5/5 [==============================] - 0s 35ms/step - loss: 2.9820 - accuracy: 0.2014
Epoch 5/100
5/5 [==============================] - 0s 35ms/step - loss: 2.9222 - accuracy: 0.3125
Epoch 6/100
5/5 [==============================] - 0s 40ms/step - loss: 2.7922 - accuracy: 0.3958
Epoch 7/100
5/5 [==============================] - 0s 41ms/step - loss: 2.6498 - accuracy: 0.4167
Epoch 8/100
5/5 [==============================] - 0s 40ms/step - loss: 2.4696 - accuracy: 0.4167
Epoch 9/100
5/5 [==============================] - 0s 40ms/step - loss: 2.1192 - accuracy: 0.4306
Epoch 10/100
5/5 [==============================] - 0s 38ms/step - loss: 1.9390 - accuracy: 0.4583
Epoch 11/100
5/5 [=

In [22]:
def response(sentence):
    sent_seq = []
    doc = nlp(repr(sentence))
    
    # split the input sentences into words
    for token in doc:
        if token.text in tokenizer.word_index:
            sent_seq.append(tokenizer.word_index[token.text])

        # handle the unknown words error
        else:
            sent_seq.append(tokenizer.word_index['<unk>'])

    sent_seq = tf.expand_dims(sent_seq, 0)
    # predict the category of input sentences
    pred = model(sent_seq)

    pred_class = np.argmax(pred.numpy(), axis=1)
    
    # choice a random response for predicted sentence
    return random.choice(intent_doc[trg_index_word[pred_class[0]]]), trg_index_word[pred_class[0]]
